In [71]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib as mplt
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from scipy import stats

from sklearn.linear_model import LogisticRegression
from IPython.core.interactiveshell import InteractiveShell
from sklearn.ensemble import RandomForestClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense

# XGB 
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
# This enables printing all lines data instead of just last line of a code box
InteractiveShell.ast_node_interactivity = "all"
# Numpy decimals to print
np.set_printoptions(precision=2)

In [53]:
df = pd.read_csv("../../../data/deep-learning-a-z/ann/Churn_Modelling.csv")
df.shape

(10000, 14)

In [54]:
shape = df.shape
df.head()
df.columns.values

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


array(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'], dtype=object)

In [55]:
df[df["Exited"]==1]["Exited"].count() / shape[0]

0.20369999999999999

## Data Preparation
### LabelEncoding and OneHotEncoding

In [56]:
dataset = df
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encoding categorical data
# Encoding the Independent Variable
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])

labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])

onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:,1:]







### Test Train Split and scaling

In [57]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Neural Networks

In [58]:
classifier = Sequential()

### Add Hidden layers

In [59]:
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu',input_dim=11))
classifier.add(Dense(units=6,kernel_initializer='uniform',activation='relu'))
classifier.add(Dense(units=8,kernel_initializer='uniform',activation='relu'))

### Add output Layer

In [60]:
classifier.add(Dense(units=1,kernel_initializer='uniform',activation='sigmoid'))

In [61]:
classifier.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

### Train

In [62]:
# verbose=0 will prevent the output bar from showing
classifier.fit(X_train,y_train,batch_size=10,epochs=50,verbose=0)

### Accuracy and Confusion Matrix

In [63]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred>0.5)



In [65]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm
accuracy = (cm[0][0] + cm[1][1])/2500
accuracy

array([[1957,   34],
       [ 361,  148]])

0.84199999999999997

## XGBOOST

### Train

In [68]:

# fit model no training data
model = XGBClassifier()
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=1, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=0, silent=True,
       subsample=1)

In [69]:

# make predictions for test data
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

### Accuracy and Confusion matrix

In [70]:
cm = confusion_matrix(y_test, y_pred)
cm
accuracy = (cm[0][0] + cm[1][1])/2500
accuracy

array([[1908,   83],
       [ 254,  255]])

0.86519999999999997

## Random Forest Classifier

### Train

In [72]:
model = RandomForestClassifier(n_estimators=25,max_features=4,min_samples_leaf=2,n_jobs=-1)
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=4, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=2,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=25, n_jobs=-1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [73]:
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [74]:
cm = confusion_matrix(y_test, y_pred)
cm
accuracy = (cm[0][0] + cm[1][1])/2500
accuracy

array([[1887,  104],
       [ 234,  275]])

0.86480000000000001